In [2]:
!pip install flask flask-cors deap numpy pandas

In [3]:
import random
import numpy as np
import pandas as pd
import threading
from deap import base, creator, tools, algorithms
from flask import Flask, request, jsonify
from flask_cors import CORS

# Initialize Flask
app = Flask(__name__)
CORS(app)

def select_best_squad(min_budget, max_budget, formation, scoring_strategy, clubboost, nationboost, currently, nogens=300, popul=1000, csv_path="final_krazi.csv"):
    # Read and preprocess dataset
    df = pd.read_csv(csv_path)
    df["Market_Value"] = pd.to_numeric(df["Market_Value"], errors="coerce")
    df = df[df["Market_Value"] > 0].reset_index(drop=True)
    if currently:
        df = df[df["Last_played"] == 2425].reset_index(drop=True)
    
    # Extract relevant data
    players = df.index.tolist()
    overall = df["overall_ratings"].tolist()
    potential = df["potential_ratings"].tolist()
    market_value = df["Market_Value"].tolist()

    # Define scoring weights
    scoring_weights = {
        "onlypotential": (0, 1),
        "futurestars": (0.15, 0.85),
        "peakplayers": (0.5, 0.5),
        "veterans": (0.85, 0.15),
        "onlyoverall": (1, 0)
    }
    
    w_overall, w_potential = scoring_weights[scoring_strategy]

    def fitness(individual):
        total_score = sum(w_overall * overall[i] + w_potential * potential[i] for i in individual)
        total_market_value = sum(market_value[i] for i in individual)

        if total_market_value > max_budget:
            return (-100000,)
        return (total_score,)

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    
    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initRepeat, creator.Individual, random.sample, players, 11)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=4)
    toolbox.register("evaluate", fitness)
    
    population = toolbox.population(n=popul)
    best_squad = tools.selBest(population, k=1)[0]
    
    selected_players = df.iloc[best_squad]
    return {
        "selected_player_ids": selected_players["id"].tolist(),
        "total_market_value": selected_players["Market_Value"].sum(),
        "average_overall": selected_players["overall_ratings"].mean(),
        "average_potential": selected_players["potential_ratings"].mean()
    }

@app.route("/select_squad", methods=["POST"])
def select_squad():
    data = request.json
    squad_result = select_best_squad(
        data.get("min_budget", 0), 
        data.get("max_budget", 1000000000),
        data.get("formation", "4-3-3"),
        data.get("scoring_strategy", "onlyoverall"),
        data.get("clubboost", False),
        data.get("nationboost", False),
        data.get("currently", False)
    )
    return jsonify(squad_result)

# Function to run Flask in the background
def run_flask():
    app.run(host="0.0.0.0", port=5000, use_reloader=False)

# Run Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.32:5000
Press CTRL+C to quit
[2025-03-19 03:55:54,244] ERROR in app: Exception on /select_squad [POST]
Traceback (most recent call last):
  File "C:\Users\Kavish\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Kavish\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Kavish\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask_cors\extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ~^^^^^^^^^^^^^^^^^
  File "C:\Users\Kavish\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.di